In [1]:
# DSPy Prompting Guide

# Quick Start
import dspy

lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')
dspy.configure(lm=lm)

# Defining a Signature
from typing import Literal

class Categorize(dspy.Signature):
    event: str = dspy.InputField()
    category: Literal['Animals', 'Dogs', 'Cats', 'Other', 'Philosophy', 'History'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

# Calling the Module
classify = dspy.Predict(Categorize)
classification = classify(event="[Origin of English Bulldog]")
print(classification)

# # Define a placeholder for the validate_category function
# def validate_category(example, prediction, trace=None):
#     return prediction.category == example.category
    
# # Optimizing Prompts


# # Optimize
# tp = dspy.MIPROv2(metric=validate_category, auto="light")
# optimized_classify = tp.compile(classify, trainset=trainset, max_labeled_demos=0, max_bootstrapped_demos=0)

# from dspy.teleprompt import *
# tp = dspy.MIPROv2(metric=validate_category, auto="light")
# optimized_classify = tp.compile(classify, trainset=trainset)

# # Saving Optimized Systems
# optimized_classify.save("optimized_event_classifier.json")


Prediction(
    category='Dogs',
    confidence=0.75
)


In [ ]:
with open("0101_events.json", 'r') as file:
    data = json.load(file)
    events = pd.DataFrame(data['events'])
    
    # Using our small model
    with dspy.context(lm=dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')):
        events['category_32_1b'], events['confidence_32_1b'] = zip(*events['description'].apply(classify_event))


In [3]:
# Generating example predictions from Llama 3.3
import os
import json
import pandas as pd

# Define a function to classify the event description
def classify_event(description):
    try:
        prediction = classify(event=description)
        return prediction.category, prediction.confidence
    except Exception as e:
        return 0, 0

with dspy.context(lm=dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')):
    # Directory containing the JSON files
    events_dir = 'events'

    # Iterate over all files in the directory
    for filename in os.listdir(events_dir):
        if filename.endswith('.json'):
            filepath = os.path.join(events_dir, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)
                events = pd.DataFrame(data['events'])
                with dspy.context(lm=dspy.LM('ollama_chat/llama3.3', api_base='http://localhost:11434')):
                    events['category'], events['confidence'] = zip(*events['description'].progress_apply(classify_event))
                # Append the results to a global dataframe
                if 'lmma_events' not in globals():
                    lmma_events = events
                else:
                    lmma_events = pd.concat([lmma_events, events], ignore_index=True)
                
                # Break if the dataframe has more than 300 rows
                if len(lmma_events) > 300:
                    print('Breaking...')
                    break

# Save the results to a CSV file
lmma_events.to_csv('llama_3_3_trainset.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'events'

In [ ]:
import csv
import dspy
from dspy.evaluate import Evaluate

# Load the trainset
trainset = []
with open('llama_3_3_trainset.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        example = dspy.Example(event=row['description'], category=row['category']).with_inputs("event")
        trainset.append(example)

# Evaluate our existing function
evaluator = Evaluate(devset=trainset, num_threads=1, display_progress=True, display_table=5)
evaluator(classify, metric=validate_category)

In [1]:

# Load our model
lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')
dspy.configure(lm=lm)

# Optimize
tp = dspy.MIPROv2(metric=validate_category, auto="light")
optimized_classify = tp.compile(classify, trainset=trainset, max_labeled_demos=0, max_bootstrapped_demos=0)

NameError: name 'dspy' is not defined